### **Products and Market Risk**

**Question 3.**

Implement the multi-step binomial method as described in Binomial Method lecture with
the following variables and parameters: stock S = 100, interest rate r = 0.05 (continuously compounded)
for a call option with strike E = 100, and maturity T = 1. European payoff.

- Use any suitable parameterization for up and down moves uS, vS.

- Plot 1: compute option value for a range of volatility [0.05, . . . , 0.80] and plot the result (volatility at axe X, and option value axe Y). Trees to have a minimum four time steps.

- Plot 2: now, set σimp = 0.2 and compute and plot the value of one option, as you increase the
number of time steps NT S = 4, 5, . . . , 50.

Hint: This is a computational problem best coded in Python. You can visualize 1-2 trees (optional) but
especially in case of Excel computation, do not provide multiple pages of individual pricing trees.

**Step 1: parameterization of the ups and downs**

In binominal model we typically consider the length of 1 time step as $\Delta t = \frac {T} {N}$, here, N is the time step

One of the commonly used model is Cox-Ross-Rubinstein, which has a parameterization set of:

$$ \mu = e^{\sigma \sqrt {\Delta t}}, \quad d = e^{- \sigma \sqrt {\Delta t}}$$

Thus making the up and down movement symmetrical and relates to volatility and time step.

**Step 2: risk neutral probability**

Binominal model assumes risk neutral, thus, the risk neutral probability is calculated as:

$$q = \frac {e^{r \Delta t} - d } {u - d}$$

Now with $u, d, q$ in place, we can now construct the price movement tree and option value tree.

**Step 3: constructing the price movement tree and calculate option value backward**

For a binominal tree with N steps, we consider the price tree as:

- Step 0 with price $S_0 = 100$

- Step 1 with two nodes of $S_{0u}$ and $S_{0d}$

- Step 2 with three nodes of $S_{0u^2}$, $S_{0ud}$ and $S_{0d^2}$

- Continuing until step N with N+1 nodes

For a european option, the option value at the time of N is:

$$C_{N,j} = max (S_{N,j} - E, 0)$$

which, S_{N,j} represent the j node of the Nth step (j ranges from 0 to N)

Then we calculate backward

$$C_{N,j} = e^{- r \Delta t} [qC_{N+1,j+1} + (1 - q) C_{N+1,j}]$$

Eventually, $C_{0,0}$ is the option price

**Step 4: **